In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
! pip uninstall -y tensorflow_datasets
! pip install tensorflow_datasets==4.4.0

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import  LabelEncoder
from tqdm.auto import tqdm
import tensorflow as tf
import dill
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import glob

# This notebook shows the easiest way to convert any dataset to TFRecords

In [ ]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")

In [ ]:
test_df = pd.DataFrame({
    'filename':[os.path.basename(name) for name in glob.glob("../input/sorghum-id-fgvc-9/test/*.png")]
})
test_df['cultivar'] = ''
test_df.to_csv("submission.csv",index=False)

In [ ]:
test_df.head(5)

In [ ]:
train_df['jpeg_image'] = [path[:-3]+"jpg" for path in train_df.image]
test_df['jpeg_image'] = [path[:-3]+"jpg" for path in test_df.filename]

# Filter entries with no image associated

In [ ]:
def image_exists(path):
    return os.path.exists(path)
train_df = train_df[[image_exists(f"../input/sorghum-id-fgvc-9/train_images/{img}") for img in train_df.image]]

In [ ]:
train_df.shape,test_df.shape

In [ ]:
test_df

# Encode cultivar

In [ ]:
le = LabelEncoder()
le = le.fit(train_df.cultivar)
dill.dump(le,open("le.dill",'wb'))
train_df["target"] = le.transform(train_df.cultivar)

In [ ]:
train_df.head(5)

# Define the dataset class

We need to save the class in a .py file and import it

In [ ]:
%%writefile fgvc_dataset.py -a

import tensorflow_datasets as tfds
import tensorflow as tf

class FGVCDataset(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('0.1.0')
    
    def _split_generators(self, dl_manager):
        arr = [
            tfds.core.SplitGenerator(name=f'train',gen_kwargs={"split":"train"}),
            tfds.core.SplitGenerator(name=f'test',gen_kwargs={"split":"test"})
        ]
        return arr
    
    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=(""),
            #disable_shuffling=True,
            features=tfds.features.FeaturesDict({
                "img": tfds.features.Image(encoding_format='jpeg'),#dtype=tf.uint8,shape=(self.WIDTH,self.HEIGHT,3),
                "name": tfds.features.Tensor(dtype=tf.string,shape=()),
                "cultivar": tfds.features.Tensor(dtype=tf.string,shape=()),
                "target": tfds.features.Tensor(dtype=tf.int32,shape=()),
            }),
        )
    
    def _generate_examples(self,**args):
        print(args)
        split = args["split"]
        
        if split == 'train':
            for i in range(len(self.train_df)):
                row = self.train_df.iloc[i]
                img = f"../input/sorghum100cultivarjpgimages512x512/train_images/{row.jpeg_image}"
                yield i, {
                    'img':img,
                    'cultivar':row.cultivar,
                    'name':row.image,
                    'target':row.target,
                }
                
        if split == 'test':
            for i in range(len(self.test_df)):
                row = self.test_df.iloc[i]
                img = f"../input/sorghum100cultivarjpgimages512x512/test/{row.jpeg_image}"
                yield i, {
                    'img':img,
                    'name':row.filename,
                    'cultivar':'',
                    'target':-1,
                }

# Images to TFRecords

In [ ]:
from fgvc_dataset import FGVCDataset

In [ ]:
%%time
data_dir='fgvc_dataset'
FGVCDataset.train_df = train_df#.iloc[:100]
FGVCDataset.test_df = test_df#.iloc[:100]

builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare() 

# Test the dataset

In [ ]:
builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare() 

In [ ]:
builder.as_dataset()

In [ ]:
train_ds = builder.as_dataset()['train']
test_ds = builder.as_dataset()['test']

In [ ]:
for x in train_ds.take(1):
    img = x['img']
    cultivar = x['cultivar']
    target = x['target']
    print(img.shape,cultivar,target)

In [ ]:
plt.imshow(img.numpy())

Check Training Notebook at  https://www.kaggle.com/tchaye59/efficientnetb0-tensorflow-baseline-tpu